In [10]:
from keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

# Import the libraries required to use convolutional neural networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the images 224 -> 128
train_generator = train_datagen.flow_from_directory(directory=("/archive/train"),
                                                    target_size=(128, 128),
                                                    color_mode="grayscale",
                                                    batch_size=6326,
                                                    class_mode="sparse",
                                                    shuffle=True,seed=27)

valid_generator = valid_datagen.flow_from_directory(directory=("/archive/val"),
                                                    target_size=(128, 128),
                                                    color_mode="grayscale",
                                                    batch_size=38,
                                                    class_mode="sparse",
                                                    shuffle=True,seed=27)


test_generator = test_datagen.flow_from_directory(directory=("/archive/test"),
                                                  target_size=(128, 128),
                                                  color_mode="grayscale",
                                                  batch_size=771,
                                                  class_mode="sparse",
                                                  shuffle=False,
                                                  seed=27)

In [ ]:
x_train, y_train = train_generator.next()
x_val, y_val = valid_generator.next()
x_test, y_test = test_generator.next()

print("X_train is: ", x_train.shape)
print("Y_train is: ", (y_train.shape))
print("X_val is: ", (x_val.shape))
print("Y_val is: ", (y_val.shape))
print("X_test is: ", (x_test.shape))
print("Y_test is: ", (y_test.shape))

In [ ]:
print(y_train[0:10])
print(y_val[0:10])
print(y_test[100:110])

In [ ]:
#Data Visualization¶

In [ ]:
labels = list(train_generator.class_indices.keys())
print(labels)

#Print an image from the dataset. NORMAL can be replaced for any other class.
ran_img_label = np.random.choice(os.listdir("/content/train/NORMAL"))
img = load_img(os.path.join("/content/train/NORMAL", ran_img_label))
plt.imshow(img)

In [ ]:
fig, axs = plt.subplots()
image = x_train[0].reshape(x_train[0].shape[0],-1)
axs.imshow(image, cmap='gray')  # imshow renders a 2D grid
plt.show()

In [ ]:
#Data Analysis

In [ ]:
import tensorflow as tf
from tensorflow import keras

#We used the cnn from the exam since we did not have much time to work in the project (We only had 2 days)
def cnn_model(input_shape, learning_rate=0.01):
  tf.keras.backend.clear_session()
  np.random.seed(0)
  tf.random.set_seed(0)

  num_classes = 4

  model = keras.Sequential()

  model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same',
                  input_shape=input_shape))

  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))


  model.add(keras.layers.Conv2D(filters=16, kernel_size=(3, 3)))
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))


  model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same'))
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.Flatten())


  model.add(keras.layers.Dense(units=512))
  model.add(keras.layers.Activation('relu'))

  model.add(keras.layers.Dense(units=num_classes))
  model.add(keras.layers.Activation('softmax'))

  optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

  model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])


  return model

model = cnn_model((128, 128, 1), learning_rate=0.01)
model.summary()

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns  # for nicer plots
sns.set(style='darkgrid')  # default style
import plotly.graph_objs as plotly

def plot_history(history):
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.xticks(range(0, len(history['loss'] + 1)))
  plt.plot(history['loss'], label="training", marker='o')
  plt.plot(history['val_loss'], label="validation", marker='o')
  plt.legend()
  plt.show()

In [ ]:
history = model.fit(
    x = x_train,
    y = y_train,
    epochs=10,
    batch_size=64,
    validation_data=(x_val, y_val),
    verbose=1
  )

history = pd.DataFrame(history.history)
plot_history(history)

In [ ]:
y_pred_cnn = model.predict(x_test)
y_pred_cnn = np.argmax(y_pred_cnn, axis = -1)
print("Real labels", y_test[100:120])
print("Predicted labels", y_pred_cnn[100:120])